In [1]:
### 기계공학과 4학년
### 20154523 강남규

## 블록 깨기 2

In [2]:
import math
import pygame

black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)

block_width = 23
block_height = 15
ball_spped =10

class Block(pygame.sprite.Sprite):
    def __init__(self, color, x, y):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface([block_width, block_height])
        self.image.fill(color)
        self.image = pygame.image.load("block"+str(i)+".png").convert()
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

class Ball(pygame.sprite.Sprite):
    speed = ball_spped
    x = 0.0
    y = 180.0
    direction = 200
    width = 10
    height = 10

    def __init__(self):
        pygame.sprite.Sprite.__init__(self)

        self.image = pygame.Surface([self.width, self.height])
        self.image.fill(red)
        self.image = pygame.image.load("hart.png").convert()
        self.image.set_colorkey(white)
        self.rect = self.image.get_rect()
        self.screenheight = pygame.display.get_surface().get_height()
        self.screenwidth = pygame.display.get_surface().get_width()

    def bounce(self, diff):
        self.direction = (180 - self.direction)%360
        #cos 부호 반전, == y 방향 반전
        self.direction -= diff

    def update(self):
        direction_radians = math.radians(self.direction)
        self.x += self.speed * math.sin(direction_radians)
        self.y -= self.speed * math.cos(direction_radians)
        self.rect.x = self.x
        self.rect.y = self.y

        if self.y <= 0:
            self.bounce(0)
            self.y = 1

        if self.x <= 0:
            self.direction = (360 - self.direction)%360
            self.x = 1

        if self.x > self.screenwidth - self.width:
            self.direction = (360 - self.direction)%360
            self.x = self.screenwidth - self.width - 1

        if self.y > 600:
            return True
        else:
            return False

class Player(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.width = 75
        self.height = 15
        self.image = pygame.Surface([self.width, self.height])
        self.image.fill(green)
        self.image = pygame.image.load("bar.png").convert()

        self.rect = self.image.get_rect()
        self.screenheight = pygame.display.get_surface().get_height()
        self.screenwidth = pygame.display.get_surface().get_width()
        self.rect.topleft = (0, self.screenheight - self.height)

    def update(self):
        pos = pygame.mouse.get_pos()
        self.rect.left = pos[0]

        if self.rect.left > self.screenwidth - self.width:
            self.rect.left = self.screenwidth - self.width


pygame.init()

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode([screen_width,screen_height])
pygame.display.set_caption('블록 꺠기')

pygame.mouse.set_visible(0)
font = pygame.font.Font('malgun.ttf', 36)
font2 = pygame.font.Font('malgun.ttf', 25)
load_sound = pygame.mixer.Sound("load.wav")
shoot_sound = pygame.mixer.Sound("shoot.wav")
point = 0
total_time = 10
start_ticks = pygame.time.get_ticks()

background = pygame.Surface(screen.get_size())
background.fill(black)
background_image = pygame.image.load("back1.png").convert()
background_position=[0,0]

blocks = pygame.sprite.Group()
balls = pygame.sprite.Group()
allsprites = pygame.sprite.Group()
player = Player()
allsprites.add(player)
ball = Ball()
allsprites.add(ball)
balls.add(ball)

top = 80
blockcount = 32
i=0

for row in range(5):
    i += 1
    for column in range(0, blockcount):
        block = Block(blue, column * (block_width+2)+1, top)
        blocks.add(block)
        allsprites.add(block)
    top += block_height+2
  
    
clock = pygame.time.Clock()
game_over = False
exit_program = False

while exit_program != True:
    clock.tick(30)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            exit_program = True

    if not game_over:
        player.update()
        game_over = ball.update()
    
    screen.blit(background_image, background_position)

    if game_over:
        text = font.render("Game Over", 1, red)
        textpos = text.get_rect(centerx = screen.get_width()/2)
        screen.blit(text, textpos)

    if pygame.sprite.spritecollide(player, balls, False):
        diff = (player.rect.left + player.width/2) - (ball.rect.left + ball.width/2)

        ball.rect.top = screen.get_height() - player.rect.height - ball.rect.height -1
        ball.bounce(diff)
        load_sound.play()
    
    deadblocks = pygame.sprite.spritecollide(ball, blocks, True)

    if len(deadblocks) > 0:
        ball.bounce(0)
        point += len(deadblocks)
        shoot_sound.play()

    if len(blocks) == 0:
        game_over = True
    
    
    #포인트에 따라 속도 증가
    if point > 32*4:
        ball_speed = 60
    elif point > 32*3:
        ball_speed = 50
    elif point > 32*2:
        ball_speed = 40
    elif point > 32*1:
        ball_speed = 30
    elif point > 16:
        ball_speed = 20
    else:
        ball_speed = 10
    
    #레벨 표시
    level = str(int(ball_speed/10))
    text = font2.render("Level : " + str(level), True, black)
    screen.blit(text, [690,40])
               
    #포인트 계산
    text = font2.render("Point : " + str(point), True, black)
    screen.blit(text, [10,40])
 
    #경과시간 계산
    elapsed_time = (pygame.time.get_ticks() - start_ticks) / 1000
    timer = font2.render("Timer : " + str(int(elapsed_time)), True, black)

    #경과 시간 표시
    screen.blit(screen, (0, 0))
    screen.blit(timer, (10, 10))
    

    allsprites.draw(screen)

    pygame.display.flip()

pygame.quit()


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


<ipython-input-2-2812e21ee854>:52: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.rect.x = self.x
<ipython-input-2-2812e21ee854>:53: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.rect.y = self.y
<ipython-input-2-2812e21ee854>:156: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  textpos = text.get_rect(centerx = screen.get_width()/2)
